# v3 - as-is from v2 but added PromptTemplate

In [1]:
# imports
import os
from dotenv import load_dotenv
from genai.schemas import GenerateParams
from genai.credentials import Credentials
from langchain.vectorstores import Chroma
from genai.extensions.langchain import LangChainInterface
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, TokenTextSplitter
from langchain.document_loaders import PDFMinerLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

## Indexing - Starts here!

In [2]:
# variables
pdf_folder_path = './data'
db_folder_path = './db_v3'

In [3]:
# define embedding function
def initEmbedFunc():
    embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    return embedding_function

In [4]:
# populate chroma db
def generateDB():
    docs = []
    for root, dirs, files in os.walk(pdf_folder_path):
        for file in files:
            if file.endswith(".pdf"):
                print(f'Reading File: {file}')
                
                # read PDF
                loader = PDFMinerLoader(os.path.join(root, file))
                documents = loader.load()

                # load the document and split it into chunks
                text_splitter = RecursiveCharacterTextSplitter(
                                    chunk_size=1000, 
                                    chunk_overlap=200,
                                    separators=["\n"]
                )
                temp = text_splitter.split_documents(documents)
                
                # append to docs
                docs += temp

    # create the open-source embedding function
    embedding_function = initEmbedFunc()

    # save to disk
    db = Chroma.from_documents(docs, embedding_function, persist_directory=db_folder_path)
    
    return db

In [5]:
db = None

if [f for f in os.listdir(db_folder_path) if not f.startswith('.')] == []:
    print("Chroma DB is empty. Generating indexes...")
    
    # generate chroma db
    db = generateDB()
else:
    print("Chroma DB is not empty.")

    # create the open-source embedding function
    embedding_function = initEmbedFunc()

    # load from disk
    db = Chroma(persist_directory=db_folder_path, embedding_function=embedding_function)

Chroma DB is not empty.


## RAG - Starts here!

In [6]:
# retrieve the watsonx.ai credentials
load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_url = os.getenv("GENAI_API", None)
creds = Credentials(api_key, api_endpoint=api_url)

In [312]:
# variables
# ibm/mpt-7b-instruct -> 3/5
# meta-llama/llama-2-7b -> 3/5
# ibm/granite-13b-sft -> 3/5
# google/ul2 -> 3.5/5
# google/flan-ul2 -> Short Answers
# google/flan-t5-xl -> Short Answers
# google/flan-t5-xxl -> Short Answers
# meta-llama/llama-2-13b-chat
# meta-llama/llama-2-13b-chat-beam
model_id = 'meta-llama/llama-2-13b-chat-beam'

In [313]:
# # generate LLM params
params = GenerateParams(
            decoding_method='greedy', 
            min_new_tokens=1,
            max_new_tokens=200,
            stream=False,
            repetition_penalty=1.2)

# params = GenerateParams(
#     decoding_method="sample",
#     max_new_tokens=200,
#     min_new_tokens=1,
#     stream=False,
#     temperature=0.55,
#     top_k=50,
#     top_p=1,
#     repetition_penalty=1.5
# )

In [314]:
# create a langchain interface to use with retrieved content
langchain_model = LangChainInterface(model=model_id, params=params, credentials=creds)

In [315]:
# create retrieval QA
qa = RetrievalQA.from_chain_type(
        llm=langchain_model,
        chain_type="stuff",
        retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 2}),
        return_source_documents=True
)

In [316]:
# generate response
def generateResponse(query, qa):    
    generated_text = qa(query)
    answer = generated_text['result']
    return answer   

## Testing - Starts here!

In [317]:
%%time
query = "Provide the steps to configure Watson Assistant in OpenPages?"
generateResponse(query, qa)

CPU times: user 40.6 ms, sys: 4.07 ms, total: 44.7 ms
Wall time: 988 ms


' To configure Watson Assistant in OpenPages, follow these steps:\n\nStep 1: Configure an assistant.\n\nStep 2: Integrate the assistant with OpenPages.'

In [318]:
%%time
query = "What is FastMap?"
generateResponse(query, qa)

CPU times: user 31.7 ms, sys: 0 ns, total: 31.7 ms
Wall time: 1.27 s


' FastMap is a tool used for data import and validation in OpenPages. It allows users to quickly and easily import large amounts of data into OpenPages applications, and validates the data against predefined profiles to ensure it meets certain standards.'

In [319]:
%%time
query = "What is the purpose of Reporting Periods?"
generateResponse(query, qa)

CPU times: user 31 ms, sys: 512 µs, total: 31.5 ms
Wall time: 983 ms


' Reporting Periods allow organizations to track their financial activities over time, providing valuable insights into their financial health and helping them make informed decisions about future investments and resource allocation.'

In [320]:
%%time
query = "What is a Role Template?"
generateResponse(query, qa)

CPU times: user 30.5 ms, sys: 416 µs, total: 30.9 ms
Wall time: 832 ms


' A Role Template is a security object used to define all aspects of application security for various groups and users within a business unit.'

In [321]:
%%time
query = "What are the different access controls available for non-participants for a standard stage within a workflow?"
generateResponse(query, qa)

CPU times: user 31.9 ms, sys: 415 µs, total: 32.3 ms
Wall time: 941 ms


' The different access controls available for non-participants for a standard stage within a workflow include "Strict", "Read", "Open", and "No Override".'

In [322]:
%%time
query = "What is the purpose of Object Reset?"
generateResponse(query, qa)

CPU times: user 36.2 ms, sys: 338 µs, total: 36.6 ms
Wall time: 729 ms


' To automatically modify objects in the IBM OpenPages with Watson repository based on rule-based operations contained in a ruleset.'

In [323]:
%%time
query = "What are the features of Operational Risk Management in OpenPages?"
generateResponse(query, qa)

CPU times: user 32.1 ms, sys: 0 ns, total: 32.1 ms
Wall time: 1.26 s


' The features of Operational Risk Management in OpenPages include Loss Events, Risk and Control Self-Assessment, Scenario Analysis, External Losses, Key Risk Indicators, and Integration with other systems.'

In [324]:
%%time
query = "What is the difference between PRE and POST position in Triggers?"
generateResponse(query, qa)

CPU times: user 32.7 ms, sys: 45 µs, total: 32.7 ms
Wall time: 828 ms


' In IBM Openpages with Watson, PRE triggers execute before the actual operation is performed, while POST triggers execute after the operation has been performed.'

In [325]:
%%time
query = "List all the System variables that could be used in expressions?"
generateResponse(query, qa)

CPU times: user 32.2 ms, sys: 320 µs, total: 32.6 ms
Wall time: 1.9 s


' You can use any system variable that has been defined in your configuration file or in a specific application. Some common system variables include $END_USER$, $TODAY$, $DAYS_FROM_NOW$, $APPLICATION_URL$, $COGNOS_URL$, $SETTING/, $TASK_VIEW_URL$.'

In [326]:
%%time
query = "What are the features of Business Continuity Management in OpenPages?"
generateResponse(query, qa)

CPU times: user 31.5 ms, sys: 299 µs, total: 31.8 ms
Wall time: 1.53 s


' Based on the provided text, some of the key features of Business Continuity Management in OpenPages include centralizing business continuity data, establishing and monitoring impact tolerance thresholds, performing business impact analyses, developing business continuity plans, and automating the association of regulations impacted by regulatory events.'

In [327]:
%%time
query = "Provide the attributes required to configure Detect Property Change Rule?"
generateResponse(query, qa)

CPU times: user 32 ms, sys: 0 ns, total: 32 ms
Wall time: 694 ms


' The attributes required to configure Detect Property Change Rule include "fields", "check.for", and "type".'

In [328]:
from langchain import PromptTemplate

# Define prompt
template = """Answer the question based on the context below. Keep the answer short and concise. Respond "Unsure about answer" if not sure about the answer.

Context: {context}

Question: {question}

Answer: """

# instantiate prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

In [329]:
# create retrieval QA
qa1 = RetrievalQA.from_chain_type(
        llm=langchain_model,
        chain_type="stuff",
        retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 7}),
        chain_type_kwargs={"prompt": prompt_template}
)

In [330]:
%%time
query = "Provide steps to configure Watson Assistant in OpenPages?"
qa1.run(query)

CPU times: user 30.1 ms, sys: 1.12 ms, total: 31.3 ms
Wall time: 3.3 s


' Based on the provided context, here are the steps to configure Watson Assistant in OpenPages:\n\nStep 1: Configure an assistant.\n\nStep 2: Integrate the assistant with OpenPages.\n\nStep 3: Define the skills that the assistant should have.\n\nStep 4: Map the inputs and outputs of the assistant.\n\nStep 5: Test the assistant to ensure it is functioning correctly.\n\nNote: These steps are based on the provided context and may vary depending on the specific requirements of the implementation.'

In [331]:
%%time
query = "What is FastMap?"
qa1.run(query)

CPU times: user 36 ms, sys: 3.95 ms, total: 39.9 ms
Wall time: 1.45 s


' FastMap is a feature within Oracle Enterprise Performance Management [EPM] Cloud that provides a simple way to import large amounts of data into EPM applications without having to write any code. It also helps to automate repetitive tasks and improve productivity.'

In [332]:
%%time
query = "What is a Reporting Period?"
qa1.run(query)

CPU times: user 30.6 ms, sys: 1.6 ms, total: 32.2 ms
Wall time: 1.53 s


' A reporting period is a defined scope of time during which financial transactions are recorded and reported on. It typically includes a start and end date, and may be divided into different sub-periods for various purposes, such as budgeting, forecasting, or financial reporting.'

In [333]:
%%time
query = "What is a Role Template?"
qa1.run(query)

CPU times: user 32.9 ms, sys: 74 µs, total: 33 ms
Wall time: 440 ms


' Unsure about answer'

In [334]:
%%time
query = "What are the different types of access controls available for non-participants for a standard stage within a workflow?"
qa1.run(query)

CPU times: user 29.2 ms, sys: 3.93 ms, total: 33.1 ms
Wall time: 3.01 s


' Based on the given context, there are four different types of access controls available for non-participants for a standard stage within a workflow:\n\n1. Strict - Only the owner of the object can view and edit it.\n2. Read - Non-participants can view the object but cannot edit it.\n3. Open - Anyone can view and edit the object.\n4. No Override - Standard access controls apply, and non-participants do not have any special access rights.'

In [335]:
%%time
query = "What is the purpose of Object Reset?"
qa1.run(query)

CPU times: user 24.5 ms, sys: 7.37 ms, total: 31.9 ms
Wall time: 485 ms


' Unsure about answer'

In [336]:
%%time
query = "What are the features of Operational Risk Management in OpenPages?"
qa1.run(query)

CPU times: user 32.2 ms, sys: 0 ns, total: 32.2 ms
Wall time: 1.67 s


' The features of Operational Risk Management in OpenPages include Loss Events, Risk and Control Self Assessments (RCSA), Scenario Analysis, External Losses, Key Risk Indicators (KRI), and Issue Management and Remediation.'

In [337]:
%%time
query = "What is the difference between PRE and POST position in Triggers?"
qa1.run(query)

CPU times: user 42.3 ms, sys: 2.72 ms, total: 45 ms
Wall time: 495 ms


' Unsure about answer'

In [338]:
%%time
query = "List the user administration permissions that can be delegated."
qa1.run(query)

CPU times: user 27.6 ms, sys: 4.42 ms, total: 32 ms
Wall time: 1.43 s


' Based on the provided context, the following user administration permissions can be delegated:\n\n• Resetting passwords for users\n\n• Locking and unlocking users\n\n• Creating users and associating them to user groups and assigning role templates.'

In [ ]:
%%time
query = "What are the System variables that could be used in expressions?"
qa1.run(query)

CPU times: user 33.2 ms, sys: 420 µs, total: 33.6 ms
Wall time: 2.93 s


' Variables that contain application data that can be used in expressions include:\n\n($END_USER$) User who is currently logged in.\n\n($TODAY$) Today’s date.\n\n($TASK_VIEW_URL$) URL to an object task view.\n\n($SYSTEM_FIELDS:TASK_VIEW_URL$) Deprecated. Use ($TASK_VIEW_URL$).\n\n($SETTING/OPENPAGES/...) Registry settings value.'